# Day 9: Smoke Basin

[https://adventofcode.com/2021/day/9](https://adventofcode.com/2021/day/9)

## Description

### Part One

These caves seem to be [lava tubes](https://en.wikipedia.org/wiki/Lava_tube). Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the _low points_ - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are _four_ low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The _risk level_ of a low point is _1 plus its height_. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore _`15`_.

Find all of the low points on your heightmap. _What is the sum of the risk levels of all low points on your heightmap?_

In [1]:
data = """2199943210
3987894921
9856789892
8767896789
9899965678""".split()


def neighbours(data, x, y):
    if x > 0:
        yield data[x - 1][y]
    if x < len(data) - 1:
        yield data[x + 1][y]
    if y > 0:
        yield data[x][y - 1]
    if y < len(data[x]) - 1:
        yield data[x][y + 1]


def low(data):
    for x in range(len(data)):
        for y in range(len(data[x])):
            v = data[x][y]
            if all(v < neighbour for neighbour in neighbours(data, x, y)):
                yield v, x, y


def part1(data):
    data = [[int(d) for d in line] for line in data]
    print("part 1", sum(v + 1 for v, _, _ in low(data)))


part1(data)
part1([s.strip() for s in open("input/day9.txt").readlines()])

part 1 15
part 1 486


<div class="alert alert-info">Pretty straightforward. Yielding x and y from low was added for part 2 but doesn't
affect the result.
</div>


**Your puzzle answer was 486.**

### Part Two

Next, you need to find the largest basins so you know what areas are most important to avoid.

A _basin_ is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height `9` do not count as being in any basin, and all other locations will always be part of exactly one basin.

The _size_ of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


The top-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


The middle basin, size `14`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


The bottom-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678


Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = 1134`.

_What do you get if you multiply together the sizes of the three largest basins?_

In [3]:
def npos(data, x, y):
    if x > 0:
        yield x - 1, y
    if x < len(data) - 1:
        yield x + 1, y
    if y > 0:
        yield x, y - 1
    if y < len(data[x]) - 1:
        yield x, y + 1

<div class="alert alert-info">Same as the neighbours function in part 1 only now I need the coordinates instead of
the values.
</div>

In [4]:
def basin(data, x, y):
    points = {(x, y)}
    v = data[x][y]
    queue = [(x1, y1) for (x1, y1) in npos(data, x, y) if data[x1][y1] != 9]
    while queue:
        x1, y1 = queue.pop()
        if (x1, y1) not in points:
            points.add((x1, y1))
            for x2, y2 in npos(data, x1, y1):
                if data[x2][y2] != 9:
                    queue.append((x2, y2))
    return len(points)

<div class="alert alert-info">I really messed up badly here: I forgot to add data as a parameter and so my code
was picking up the global of the same name and it took forever to spot. 😞
</div>

In [6]:
import heapq


def part2(data):
    data = [[int(d) for d in line] for line in data]
    a, b, c = heapq.nlargest(3, [basin(data, x, y) for _, x, y in low(data)])
    print("Part 2", a * b * c)


part2(data)
part2([s.strip() for s in open("input/day9.txt").readlines()])

Part 2 1134
Part 2 1059300


**Your puzzle answer was 1059300.**

<div class="alert alert-info">Finally just count the size of each basin and pull out the top 3.
</div>